In [2]:
import pandas as pd
import numpy as np
import scipy.signal
import seaborn as sns
import matplotlib.ticker as ticker
import itertools
import math

In [ ]:
path = "/home/francisco/tps/datos/tp2/"
load_path = path + "{}.hdf"

electrodos = [8, 44, 80, 131, 185]

N_P = 10
N_S = 10
pacientes_P = []
pacientes_S = []
for load_name, N, dest, offset in [("P", N_P, pacientes_P, 0), ("S", N_S, pacientes_S, 10)]:
    for i in range(1, 1 + N):
        paciente = load_name + "{:02d}".format(i)
        df_ = pd.read_hdf(load_path.format(paciente))
        df_ = df_.loc[offset + i-1,:,electrodos,:]
        dest.append(df_)


In [ ]:
# Calcular N para hacer la transformación simbólica
min_value = 2**32
max_value = -2**32
max_n_sigma = (-1, None)

for pacientes in [pacientes_P, pacientes_S]:
    for df_paciente in pacientes:
        for idx, df_ in df_paciente.groupby(["paciente", "epoch", "sensor"]):
            s = df_["valores"]
            min_value = min(s.min(), min_value)
            max_value = max(s.max(), max_value)
            max_n_sigma = max((len(s), s.std()), max_n_sigma)
            
print(min_value, max_value, max_n_sigma)


In [ ]:
N = 100 * math.ceil((max_value - min_value) / (max_n_sigma[1] * max_n_sigma[0] ** (-1/3)))
print(N)
step = (max_value - min_value) / (N-1)
print(step)

In [ ]:
def transformacion_simbolica(v):
    simbolos = (v - min_value) // step
    simbolos[simbolos == N] = N - 1
    return simbolos
    
for pacientes in [pacientes_P, pacientes_S]:
    for i, df_p in enumerate(pacientes):
        print("Paciente", i, end="\r")
        df_p["simbolo"] = df_p.groupby(["paciente", "epoch", "sensor"]).apply(transformacion_simbolica)

In [ ]:
pacientes_P[0].head()

In [ ]:
def calcular_informacion(df_):
    s = df_["simbolo"]
    p_simbolos = [s[s == simb].count() / len(s) for simb in range(N)]
    return -sum([p * np.log(p) if p != 0 else 0 for p in p_simbolos])

entropias_P = []
entropias_S = []
for pacientes, entropias in [(pacientes_P, entropias_P), (pacientes_S, entropias_S)]:
    for i, df_p in enumerate(pacientes):
        print(i)
        s_e = df_p.groupby(["paciente", "epoch", "sensor"]).apply(calcular_informacion)
        entropias.append(s_e)


In [ ]:
prom_entropias_P = []
prom_entropias_S = []
for entropias, prom_entropias in [(entropias_P, prom_entropias_P), (entropias_S, prom_entropias_S)]:
    for s in entropias:
        prom_entropias.append(s.groupby(["sensor"]).mean())

In [ ]:

df_punto_a = pd.DataFrame({
    "Entropias": [x for s in (prom_entropias_P + prom_entropias_S) for x in s.values],
    "Capacidad cognitiva": ["Reducida" for s in prom_entropias_P for x in s.values] + ["Normal" for s in prom_entropias_P for x in s.values],
    "Sensor": [x for s in (prom_entropias_P + prom_entropias_S) for x in s.index]
})

sns.violinplot(x="Sensor", y="Entropias", hue="Capacidad cognitiva", data=df_punto_a, 
               split=True, palette="Set2", inner="stick", cut=0)
sns.plt.show()

In [ ]:
pacientes_P[0].describe()

In [ ]:
tiempos_promedios_P = []
tiempos_promedios_S = []

for tiempos, pacientes in [(tiempos_promedios_P, pacientes_P), (tiempos_promedios_S, pacientes_S)]:
    for df_p in pacientes:
        s = df_p["valores"].groupby(["tiempo"]).mean()
        s = transformacion_simbolica(s)
        print(s.describe())

# Esto no sirve porque queda todo con el mismo símbolo

In [ ]:
l1 = pacientes_P[:1]
l2 = pacientes_S[:1]

sns.plt.show()
a, b = [], []
for k, e in enumerate(electrodos):
    print("Electrodo", k + 1, "de", len(electrodos))
    for df1 in l1:
        p1 = df1.index.values[0][0]
        aux = df1.loc[p1,:2,e,:]["simbolo"]
        total = aux.shape[0]
        for i, t1 in enumerate(aux):
            if i % 50 == 0:
                print(i, "de", total, end="\r")
            p2 = df2.index.values[0][0]
            for t2 in df2.loc[p2,:2,e,:]["simbolo"]:
                a.append(int(t1))
                b.append(int(t2))
        print()
    x = np.array(a)
    y = np.array(b)

    sns.jointplot(x, y, color="k", xlim=[0, 1900], ylim=[0, 1900])
    sns.plt.show() 

In [ ]:
a, b = [1,2,3], [4,15,6]
x = np.array(a)
y = np.array(b)

sns.jointplot(x,y)
sns.plt.show()